In [ ]:
!kaggle datasets download -d masterokay/trip-best
!unzip --q ./trip-best.zip

cp_path = "./"

job_worker_name = "g_t_1e5aug_random_building"

!pip uninstall -y fashion_nets
!pip uninstall -y fashion_datasets

In [ ]:
#@title Pip Installs & Imports
#!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git
try:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
except:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
from fashionnets.train_jobs.loader.checkpoint_loader import load_latest_checkpoint

from fashionnets.models.layer.Augmentation import compose_augmentations
  

In [ ]:
#@title Prepare Environment. Download Datasets
from fashionnets.train_jobs.loader.job_loader import add_back_bone_to_train_job

environment, training_job_cfg = prepare_environment(job_worker_name, debugging=False)
import kaggle #<- Requires Secrets, therefore prepare_environment needs to be run before
kaggle.api.authenticate()

kaggle_downloader = kaggle.api.dataset_download_files #<- sadly needs to be injected

train_job = load_job_settings(environment=environment, training_job_cfg=training_job_cfg, kaggle_downloader=kaggle_downloader)

job_settings = add_back_bone_to_train_job(environment=environment, **training_job_cfg)

In [ ]:
from fashionnets.evaluate.EvaluateDeepFashion import Evaluate
from fashiondatasets.deepfashion1.DeepFashion1CBIR import DeepFashion1CBIR
from fashionnets.evaluate.helper.evaluate_helper_loader import load_model
from fashionnets.evaluate.helper.evaluate_helper_loader import load_dataset

In [ ]:
modelle = load_model(cp_path, train_job)
back_bone_modell = modelle["quadtruplet"].back_bone

datasets = load_dataset(model=back_bone_modell, job_settings=job_settings)

assert sorted(datasets.keys()) == sorted(modelle.keys())

In [ ]:
results = {}

In [ ]:
from tqdm.auto import tqdm 

steps = None
losses = list(modelle.keys())

for loss_fn_str in tqdm(losses, desc="Calc Losses"):
  print(loss_fn_str)
  model = modelle[loss_fn_str]
  loss_dataset = datasets[loss_fn_str]

  if loss_fn_str not in results.keys():
    results[loss_fn_str] = {}

  for ds_name, ds in loss_dataset.items():
    if ds_name in results[loss_fn_str].keys():
      continue
    print(ds_name)
    loss_ =model.evaluate(ds, steps=steps)
    results[loss_fn_str][ds_name] = loss_

    print(results)

In [ ]:
results
